# **<u>06.미니 배치와 데이터 로드(Mini Batch and Data Load)</u>**
* 선형회귀에 한정되지 않는 내용임. 
* **데이터를 로드하는 방법**과 `미니배치경사하강법`에 대해서 학습

## **1. 미니 배치와 배치 크기(Mini Batch and Batch Size)**

In [ ]:
x_train=torch.FloatTensor([[73,80,75],
                           [93,88,93],
                           [89,91,90],
                           [96,98,100],
                           [73,66,70]])
y_train=torch.FloatTensor([[152],[185],[180],[196],[142]])

* 샘플의 개수는 5개
* 전체 데이터를 하나의 행렬로 선언하여 전체 데이터에 대해서 경사하강법을 수행
* 데이터가 수십만개 이상이라면 전체 데이터에 대해서 경사 하강법을 수행하는 것은 매우 느리며, 많은 계산량 필요
* 전체 데이터를 **더 작은 단위**로 나누어서 학습하는 `미니 배치(Mini Batch)` 개념 등장
  1. 전체 데이터를 미니배치 단위로 나눔. 
  2. 미니배치만큼의 데이터를 로드, 해당 데이터에 대한 비용(`cost`)를 계산하고, `경사하강법`을 수행함. 
    * 미니배치의 개수만큼 경사하강법을 수행해야 전체 데이터가 한 번 전부 사용 (`1 epoch`)
    * 미니배치의 개수는 미니배치의 크기에 따라 달라지며, 미니배치의 크기를 `batch size`라고 함. 
      - `batch size`는 **2의 제곱수**를 사용
      - CPU와 GPU의 메모리가 2의 배수이므로 데이터 송수신의 효율을 높이기 위함.
  3. 미니배치의 개수만큼 2를 반복
  4. 전체 데이터에 대한 학습이 1회 끝나면 1 `에포크(epoch)`가 끝남. 
    * `epoch`: 전체 훈련 데이터가 학습에 한번 사용된 주기를 말함. 
* 경사하강법 분류
  * **배치 경사 하강법**
    - 전체 데이터에 대해서 한번에 경사 하강법을 수행하는 방법
    - 전체 데이터를 사용하므로 가중치 값이 최적값에 수렴하는 과정이 안정적이지만, 계산량이 많음. 
  * **미니배치 경사 하강법**
    - 미니 배치 단위로 경사하강법을 수행하는 방법
    - 전체 데이터의 일부만을 보고 수행하므로 최적값으로 수렴하는 과정에서 헤맬 수 있지만, 훈련속도가 빠름.

## **2. 이터레이션(Iteration)**
* 한 번의 `epoch` 내에서 이루어지는 매개변수인 가중치 `w`와 편향 `b`의 업데이트 횟수
* (예)
  - Total Data = 2000
  - Batch Size = 200
  - Iteration = (Total Data) / (Batch Size) = 10 

## **3. 데이터 로드하기(Data Load)**
* pytorch에서는 데이터를 쉽게 다룰 수 있도록 데이터셋(DataSet)과 데이터로더(DataLoader)를 제공. 
  * 다음의 기능을 제공
    - 미니 배치 학습
    - 데이터 셔플
    - 병렬처리 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#텐서 데이터셋
from torch.utils.data import TensorDataset
# 데이터 로더
from torch.utils.data import DataLoader

In [13]:
x_train=torch.FloatTensor([[73,80,75],
                           [93,88,93],
                           [89,91,90],
                           [96,98,100],
                           [73,66,70]])
y_train=torch.FloatTensor([[152],[185],[180],[196],[142]])

In [14]:
dataset=TensorDataset(x_train,y_train)
dataloader=DataLoader(dataset,batch_size=2,shuffle=True)

* DataLoader은 기본적으로 2개의 인자를 입력받음. 
  * 데이터셋
  * 미니배치의 크기: 2의 배수 
  * shuffle: `epoch`마다 데이터셋을 섞어서 데이터가 학습되는 순서를 바꿈. 

In [5]:
model=nn.Linear(3,1)
optimizer=torch.optim.SGD(model.parameters(),lr=1e-5)

In [16]:
nb_epochs=100
for epoch in range(nb_epochs+1):
    for batch_idx, samples in enumerate(dataloader):
        print(batch_idx)
        print(samples)
        x_train,y_train=samples

        #예측값, 비용
        prediction=model(x_train)
        cost=F.mse_loss(prediction,y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(epoch,nb_epochs,batch_idx+1,len(dataloader),cost.item()))
        

0
[tensor([[89., 91., 90.],
        [73., 80., 75.]]), tensor([[180.],
        [152.]])]
Epoch    0/100 Batch 1/3 Cost: 0.020022
1
[tensor([[73., 66., 70.],
        [93., 88., 93.]]), tensor([[142.],
        [185.]])]
Epoch    0/100 Batch 2/3 Cost: 4.758947
2
[tensor([[ 96.,  98., 100.]]), tensor([[196.]])]
Epoch    0/100 Batch 3/3 Cost: 3.040430
0
[tensor([[ 89.,  91.,  90.],
        [ 96.,  98., 100.]]), tensor([[180.],
        [196.]])]
Epoch    1/100 Batch 1/3 Cost: 0.275865
1
[tensor([[73., 66., 70.],
        [73., 80., 75.]]), tensor([[142.],
        [152.]])]
Epoch    1/100 Batch 2/3 Cost: 3.730054
2
[tensor([[93., 88., 93.]]), tensor([[185.]])]
Epoch    1/100 Batch 3/3 Cost: 1.953366
0
[tensor([[93., 88., 93.],
        [73., 66., 70.]]), tensor([[185.],
        [142.]])]
Epoch    2/100 Batch 1/3 Cost: 1.724007
1
[tensor([[73., 80., 75.],
        [89., 91., 90.]]), tensor([[152.],
        [180.]])]
Epoch    2/100 Batch 2/3 Cost: 1.970386
2
[tensor([[ 96.,  98., 100.]]), tensor([

In [17]:
# 임의의 입력 [73,80,75]를 선언
new_var=torch.FloatTensor([[73,80,75]])
# 입력한 값 [73,80,75]에 대해서 예측값 y를 리턴, pred_y에 저장
pred_y=model(new_var)
print('훈련 후 입력이 73,80,75일때의 예측값 :',pred_y)

훈련 후 입력이 73,80,75일때의 예측값 : tensor([[152.5696]], grad_fn=<AddmmBackward0>)
